In [107]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

%matplotlib inline

import io, os, glob, gc
import timeit
from nbformat import current

HISTOGRAM_BITS_PER_COLOR = 2

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)

execute_notebook('cuav.ipynb')

ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_SUGGESTED_THRESHOLD = 35

SHOW_DUPES = True
#MIN_DEDUPE_REGIONS = 0
#DEDUPE_REGIONS = False
#DDUPE_ACCURACY_BY = 10
#MIN_REGION_SCORE = 0

In [108]:
def ProcessFile(image):
        bin = InitBin(image)
        lut = InitLUT(bin)
        qimage = quantize(image, lut)

        hist = BuildHistogram(qimage)
        mask = HistogramThreshold(qimage, hist)
        print 'Histogram Count Threshold: %i' % HISTOGRAM_COUNT_THRESHOLD

        regions = AssignRegions(mask)
        ScoreRegions(regions, qimage, hist)
        
        print "Total Regions Before Pruning: %i\n" % regions.num_regions
        
        PruneRegionsByThreshold(regions)
        PruneRegionsByMean(regions)
        PruneRegionsByDupe(regions)
        PruneRegionsByProximity(regions)
        
        del bin,lut,qimage,hist,mask
        gc.collect()
        
        return regions

In [109]:
#from guppy import hpy
#hp = hpy()
#before = hp.heap()

## critical section here
#raw = cv2.imread('batch/few/1.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
#regions = ProcessFile(image)

#after = hp.heap()
#leftover = after - before
##import pdb; pdb.set_trace()

#print leftover.byclass

In [110]:
#raw = cv2.imread('batch/od/image_car_full.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)

#%timeit ProcessFile(image)
#%prun regions = ProcessFile(image)

In [111]:
#for dirname, dirnames, filenames in os.walk('batch/few/'):
#    for filename in sorted(filenames):
#        print(os.path.join(dirname, filename))


In [112]:
PrintSettings()

raw_files_dir = 'batch/test/'
proc_files_dir = raw_files_dir + 'results/'

if not os.path.exists(proc_files_dir):
    os.makedirs(proc_files_dir)
    
for f in sorted(glob.glob(raw_files_dir + '*.*')):
    file = f.replace(raw_files_dir,'')
    if file.lower().endswith('.jpg') or file.lower().endswith('.png'):
        print file
        raw = cv2.imread(f)
        image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
        
        regions = ProcessFile(image)
        
        pruned_image = np.copy(image)
        MarkRegions(pruned_image, regions)
        bgr = cv2.cvtColor(pruned_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(proc_files_dir + file, bgr)
        print '\n'
    else:
        print 'Skipping %s' % file

Adaptive Threshold Disabled with count set at 35 

HISTOGRAM_BITS_PER_COLOR = 2
NUM_BINS = 4
COLOR_SPECTRUM = 65
ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_CEILING_THRESHOLD = 1500
HISTOGRAM_FLOOR_THRESHOLD = 50
SHOW_DUPES = True
DEDUPE_REGIONS = True
MIN_DEDUPE_REGIONS = 5
MIN_REGION_SCORE = 0

raw2015103000291033Z.png
InitBin:
image height=2048,width=2448,depth=3
min_r=18, max_r=255, bin_spacing.r=60
min_g=18, max_g=255, bin_spacing.g=60
min_b=24, max_b=255, bin_spacing.b=58

REGION_MERGE_PIXEL_DISTANCE = 20
Histogram Count Threshold: 34
Total Regions Before Pruning: 127

Prune Common Regions (below threshold):
  Min Score: 0
  Regions Above Threshold (Active): 22
Prune Regions Exceeding Mean:
  Mean: 20
  Regions Below Mean (Active): 9
  [id]: [3, 6, 8, 9, 10, 11, 12, 13, 15]
Prune Duplicated Regions:
  Accuracy: 1
  Regions Deduped (Active): 8
  [id]: [3, 6, 8, 9, 10, 11, 13, 15]
Pruned Clustered Regions:
  Radius of Separation: 200
  Regions Isolated (Active): 4
  [id]: [3, 6,

In [113]:
%sx ipython nbconvert "oddiagnostic.ipynb"

['[NbConvertApp] Converting notebook oddiagnostic.ipynb to html',
 '[NbConvertApp] Writing 202372 bytes to oddiagnostic.html']

In [114]:
%sx mv "oddiagnostic.html" $proc_files_dir

[]